<a href="https://colab.research.google.com/github/LucianoRodriguez0764/ipadapter/blob/main/Ipadapter_faceID_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install insightface
!pip install onnxruntime
!pip install diffusers
!pip install git+https://github.com/tencent-ailab/IP-Adapter.git
!pip install einops

In [ ]:
!wget -O ip-adapter-faceid_sd15.bin https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin?download=true

In [ ]:
import torch
import cv2
from insightface.app import FaceAnalysis


app = FaceAnalysis(name="buffalo_l", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

#image = cv2.imread("face3.png")
#faces = app.get(image)

#faceid_embeds = torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)


In [137]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter.ip_adapter_faceid import IPAdapterFaceID

base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
ip_ckpt = "ip-adapter-faceid_sd15.bin"
device = "cuda"

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import os
import glob
import cv2

# Set the directory containing images
image_path = 'images/face'

imgs = glob.glob(os.path.join(image_path, '*.*'))
#imgs += glob.glob(os.path.join('images/face_2', '*.*'))
#imgs += glob.glob(os.path.join('images/face_3', '*.*'))
# Collect all embeddings
embeddings = []

for i in range(len(imgs)):
    print(imgs[i])
    image = cv2.imread(imgs[i])
    faces = app.get(image)

    # Check if a face is detected
    if faces:
        faceid_embed = torch.from_numpy(faces[0].normed_embedding).unsqueeze(0)
        embeddings.append(faceid_embed)
    else:
        print("face not detected")

# Calculate the average embedding for the face ID

weights = [3,1]
weights = [elem/sum(weights) for elem in weights]
#print(weights)

if embeddings:

    ### weights
    total = torch.zeros_like(embeddings[0])
    for i in range(len(embeddings)):
        if len(weights)==len(embeddings):
            total.add_(embeddings[i]*weights[i])
        else:
            total.add_(embeddings[i]*(1/len(embeddings)))
    faceid_embeds = total / len(embeddings[i])

    ###  mean weights
    #faceid_embeds = torch.mean(torch.stack(embeddings), dim=0)

    print("Collected face ID embedding")
else:
    print("No faces detected in the images provided.")


In [158]:
from IPython.display import display
from ipywidgets import HBox, Image
import random
random_seed = random.randint(1, 9999999)
# load ip-adapter
ip_model = IPAdapterFaceID(pipe, ip_ckpt, device)
ip_model.set_scale(1)
# generate image
prompt = ""
prompt += "photo of a 20 years old girl, red natural hair"
prompt += ""

negative_prompt = "nsfw, lowres, bad anatomy, worst quality, low quality, blurry, extra arms, extra limbs, extra fingers, bad nose"

random_seed = 8097166
print("Seed: ", random_seed)

images = ip_model.generate(
    prompt=prompt, negative_prompt=negative_prompt, faceid_embeds=faceid_embeds, num_samples=2, width=512, height=512, num_inference_steps=50, seed=random_seed,
    guidance_scale=3
)
'''
images.extend(ip_model.generate(
    prompt=prompt, negative_prompt=negative_prompt, faceid_embeds=faceid_embeds, num_samples=1, width=512, height=512, num_inference_steps=30, seed=random_seed,
    guidance_scale=2
))
'''



# Assuming images is a list of PIL Images
image_widgets = [Image(value=img._repr_png_()) for img in images]

# Display the images in a row
display(HBox(image_widgets))

Seed:  8097166


  0%|          | 0/300 [00:00<?, ?it/s]